In [ ]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, \
    load_robot_execution_failures
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

In [ ]:
print(timeseries.head())

In [ ]:
print(y.head())

In [ ]:
import matplotlib.pyplot as plt
timeseries[timeseries['id'] == 3].plot(subplots=True, sharex=True, figsize=(10,10))
plt.show()

In [ ]:
timeseries[timeseries['id'] == 21].plot(subplots=True, sharex=True, figsize=(10,10))
plt.show()

## For extracting all features with tsfresh

In [ ]:
from tsfresh import extract_features
# DataFrame extracted_features with more than 1200 different extracted features 
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

In [ ]:
# We will now remove all NaN values that were created but cannot be used on the given data 

from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

impute(extracted_features)

# We select only relevant features 
features_filtered = select_features(extracted_features, y)

In [ ]:
# Extract, impute and filter at the same time 
from tsfresh import extract_relevant_features

features_filtered_direct = extract_relevant_features(timeseries, y,
                                                     column_id='id', column_sort='time')

In [1]:
allo

NameError: name 'allo' is not defined

In [47]:
%load_ext autoreload
%autoreload 2
from pca_everyone_svr_bpd import *

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"
iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
uniq_subjectid = np.unique(vTraiSubjectId)
enc = OneHotEncoder(handle_unknown='ignore').fit(uniq_subjectid.reshape(-1,1))
print(uniq_subjectid.reshape(-1,1).shape)
print(enc)
temp = enc.transform([[1004]]).toarray()
print(temp.shape)
temp2 = np.repeat(temp,61,axis=0)
temp2.shape
#temp3 = np.random.rand(61,50)
#temp4 = np.concatenate((temp3,temp2),axis=1)
#print(temp4.shape)

NameError: name 'vTraiSubjectId' is not defined

In [48]:
print(len(vTraiSubjectId))
print(type(vTestMeasurementId))

sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/ResiVecEveryoneSVRFold4"
iNeighbors=None
pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01')

1386
<class 'numpy.ndarray'>
aaaaaaaaaaaaaaaaaaaaaaaaaaahhhhhhhhhhhhggggggggggggggg
----- 1004----- 
Using SVR
vMNTraiPCA_subjectid shape :  (63, 65)
----- 1006----- 
Using SVR
vMNTraiPCA_subjectid shape :  (28, 65)
----- 1007----- 
Using SVR
vMNTraiPCA_subjectid shape :  (219, 65)
----- 1019----- 
Using SVR
vMNTraiPCA_subjectid shape :  (35, 65)
----- 1020----- 
Using SVR
vMNTraiPCA_subjectid shape :  (154, 65)
----- 1023----- 
Using SVR
vMNTraiPCA_subjectid shape :  (82, 65)
----- 1032----- 
Using SVR
vMNTraiPCA_subjectid shape :  (139, 65)
----- 1034----- 
Using SVR
vMNTraiPCA_subjectid shape :  (30, 65)
----- 1038----- 
Using SVR
vMNTraiPCA_subjectid shape :  (164, 65)
----- 1039----- 
Using SVR
vMNTraiPCA_subjectid shape :  (101, 65)
----- 1043----- 
Using SVR
vMNTraiPCA_subjectid shape :  (25, 65)
----- 1044----- 
Using SVR
vMNTraiPCA_subjectid shape :  (57, 65)
----- 1048----- 
Using SVR
vMNTraiPCA_subjectid shape :  (71, 65)
----- 1049----- 
Using SVR
vMNTraiPCA_subjectid shape

In [53]:

%load_ext autoreload
%autoreload 2
from pca_everyone_svr_bpd import *

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/ivectors_Testing_Fold0/ivector.scp" 
iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)
print(vTraiPCA.shape)

# /home/mpgill/BeatPD/BeatPD-CLSP-JHU/pca_everyone_svr_bpd.py 

# sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
# sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"


sOut="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/resiVecEveryoneSVR_Fold1"
sKernel='linear' 
fCValue=0.00000000200000 
fEpsilon=0.1


pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(1145, 50)
aaaaaaaaaaaaaaaaaaaaaaaaaaahhhhhhhhhhhhggggggggggggggg
----- 1004----- 
Using SVR
vTraiPCA.shape  (1145, 50)
vTraiPCA_subjectid.shape :  (63, 50)
vMNTraiPCA_subjectid shape :  (63, 63)


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [42]:
from get_final_scores_accuracy import get_final_scores_SVR 

sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/"
bKnn=False
bSVR=False
bEveryoneSVR=True

get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR)

Looking for folder :  resiVecEveryoneSVR_Fold\d
lResxFolders :  ['/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold0', '/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold2', '/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold1', '/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold4', '/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold3']
lObjsFiles :  ['/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold0/objs_everyone_50_kernel_linear_c_0.02_eps_0.01.pkl', '/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold2/objs_everyone_50_kernel_linear_c_0.02_eps_0.01.pkl', '/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold1/objs_everyone_50_kernel_linear_c_0.02_eps_0.01.pkl', '/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold4/objs_everyone_50_kernel_linear_c_0.02_eps_0.01.pkl', '/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecEveryoneSVR_Fold3/objs_everyone_50_kernel_linear_c_0.02_eps_0.01.pkl']
Components found :  ['50'

In [ ]:
salut = ['100001_test_5163afe8-a6b0-4ea4-b2ba-9b4501dd5912_10040',
 '100002_test_9152519b-4b57-43be-963c-dd7218495001_10040',
 '100003_test_c7312d73-cb34-4025-b8b8-5299b4033e2f_10040',
 '100004_test_cc730391-146b-420f-9255-c3185061f178_10040',
 '100005_test_cc7b822c-e310-46f0-a8ea-98c95fdb67a1_10041',
 '100006_test_5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a_10041',
 '100007_test_19a3e9ea-fce1-40b7-9457-2618970beb7b_10041',
 '100008_test_e2973da8-1250-4a7c-98d5-b165570a8aeb_10041',
 '100009_test_dc90dc36-b4e5-43ec-b3e8-47c39c763c71_10041']

salut2= ['100001_test_5163afe8-a6b0-4ea4-b2ba-9b4501dd5912_hbv0401',
 '100002_test_9152519b-4b57-43be-963c-dd7218495001_hbv0401',
 '100003_test_c7312d73-cb34-4025-b8b8-5299b4033e2f_hbv0401',
 '100004_test_cc730391-146b-420f-9255-c3185061f178_hbv0401',
 '100005_test_cc7b822c-e310-46f0-a8ea-98c95fdb67a1_hbv0411',
 '100006_test_5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a_hbv0411',
 '100007_test_19a3e9ea-fce1-40b7-9457-2618970beb7b_hbv0411',
 '100008_test_e2973da8-1250-4a7c-98d5-b165570a8aeb_hbv0412',
 '100009_test_dc90dc36-b4e5-43ec-b3e8-47c39c763c71_hbv0412']


sPattern=r'(?<=[_])(1[0-9]{3}(?<=\d)(?!=$)|hbv[0-9]{3})'

vTraiSubjectId = [re.findall(sPattern, fileName)[0] for fileName in salut]

# vTraiSubjectId = np.array(([x[-5:-1] for x in np.array(salut)]))
print(vTraiSubjectId)

vTraiSubjectId2 = [re.findall(sPattern, fileName)[0] for fileName in salut2]
print(vTraiSubjectId2)